# 2.6 Segmentación Básica\n\n- Objetivo: agrupar clientes/productos para análisis y priorización.\n- Prerrequisitos: métricas básicas calculadas.\n- Ejercicios: categorías de margen, unidades por segmento, ticket vs global.\n- Reto: RFM simplificado con etiquetas.\n- Errores comunes: etiquetar sin limpiar datos; CASE incompletos; mezclar métricas y etiquetas.

In [ ]:
-- Segmento vs volumen de unidades
SELECT c.segmento, SUM(fv.cantidad) AS unidades
FROM dbo.fact_ventas fv
JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
GROUP BY c.segmento;

In [ ]:
-- Clasificación simple de productos por margen (alto, medio, bajo)
SELECT producto_id, nombre,
       CASE WHEN (precio_unitario - costo_unitario) >= 30 THEN 'ALTO'
            WHEN (precio_unitario - costo_unitario) >= 15 THEN 'MEDIO'
            ELSE 'BAJO' END AS categoria_margen
FROM dbo.dim_productos;

🟢 Ejercicio: Tabla derivada con unidades y margen por producto y categoría de margen.

In [ ]:
WITH prod AS (
  SELECT p.producto_id, p.nombre, (p.precio_unitario - p.costo_unitario) AS margen_u,
         CASE WHEN (p.precio_unitario - p.costo_unitario) >= 30 THEN 'ALTO'
              WHEN (p.precio_unitario - p.costo_unitario) >= 15 THEN 'MEDIO'
              ELSE 'BAJO' END AS categoria_margen
  FROM dbo.dim_productos p
)
SELECT categoria_margen, SUM(fv.cantidad) AS unidades, SUM(fv.cantidad*margen_u) AS margen_total
FROM prod
LEFT JOIN dbo.fact_ventas fv ON prod.producto_id = fv.producto_id
GROUP BY categoria_margen;

🟠 Ejercicio: Clientes con ticket promedio > ticket promedio global (JOIN vs subconsulta).

In [ ]:
WITH global AS (
  SELECT CAST(SUM(fv.cantidad*p.precio_unitario*(1 - fv.descuento_pct/100.0)) AS DECIMAL(10,2)) / COUNT(*) AS ticket_global
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
), cliente AS (
  SELECT c.cliente_id, c.segmento,
         CAST(SUM(fv.cantidad*p.precio_unitario*(1 - fv.descuento_pct/100.0)) AS DECIMAL(10,2)) / COUNT(*) AS ticket_cliente
  FROM dbo.fact_ventas fv
  JOIN dbo.dim_clientes c ON fv.cliente_id = c.cliente_id
  JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
  GROUP BY c.cliente_id, c.segmento
)
SELECT cliente_id, segmento, ticket_cliente
FROM cliente CROSS JOIN global
WHERE ticket_cliente > ticket_global;

🔴 Reto: Crear segmentación RFM simplificada (Recency: días desde última compra, Frequency: días distintos con venta, Monetary: margen total) y asignar etiqueta compuesta.

Errores comunes: etiquetar antes de limpiar datos; usar CASE anidados sin cubrir todos los rangos; no separar métricas de etiquetas.

# Créditos

Este material fue revisado y enriquecido parcialmente mediante asistencia de IA (OpenAI y Claude); la validación y decisiones editoriales finales son humanas.

---
## Navegación
[⬅️ Anterior](05_kpis_avanzados.ipynb) | [Siguiente ➡️](07_optimizar_consultas_basico.ipynb)
---
